# Clean Data (Pt. 2)
* **Filename**: clean_race_data.ipynb
* **Author**: Angelina Li
* **Date**: 08/22/2018
* **Description**: Contribute additional racial demographic data to existing dataset
* **Input**: master_dataset.csv, independently collected data
* **Output**: Person-level (leads + contestants) dataset (master_dataset) with full racial demographic data flags; U.S. yearly racial demographics data.

### Sections
* [Helper Datasets](#hand-coding-input)
* [U.S. Demographic Data](#us-demographics)

In [1]:
import re
import pandas as pd
import os

In [2]:
# name key directories

input_dir = "../input"
intermed_dir = "../intermediate"
output_dir = "../output"

<a id="hand-coding-input"></a>
### Create Helper Datasets
* To speed up the process of manually coding race-based flags for all Bachelor/ette candidates and leads, I'm going to assume that the karenx dataset found a complete set of POC for the seasons it examined. For those years, I will just independently categorize the people that the karenx dataset already identified.
* **Objective: Save two datasets: 1. People in the karenx dataset; 2. People in years not in the karenx dataset + leads **

In [3]:
# import the master dataset
master_path = os.path.join(intermed_dir, "master_dataset.csv")
df_master = pd.read_csv(master_path)
df_master.head()

,cid,d1,d10,d2,d3,d4,d5,d6,d7,d8,...,e8,e9,lead,lead_flag,name,num_contestants,poc_flag,season,show,year
0,BA_1_ALEX_M_L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Alex Michel,True,Alex Michel,25,NaN,1,Bachelor,2002
1,BA_01_AMANDA_M,NaN,NaN,D5,D1,D1,D1,D1,NaN,NaN,...,NaN,NaN,Alex Michel,False,Amanda M,25,NaN,1,Bachelor,2002
2,BA_01_TRISTA_R,NaN,NaN,D5,D1,D1,D1,D1,NaN,NaN,...,NaN,NaN,Alex Michel,False,Trista R,25,NaN,1,Bachelor,2002
3,BA_01_SHANNON_O,NaN,NaN,D5,D1,D1,D1,NaN,NaN,NaN,...,NaN,NaN,Alex Michel,False,Shannon O,25,NaN,1,Bachelor,2002
4,BA_01_KIM_X,NaN,NaN,D5,D4,D1,NaN,NaN,NaN,NaN,...,NaN,NaN,Alex Michel,False,Kim X,25,NaN,1,Bachelor,2002


In [4]:
# get people in the karenx dataset
df_kx_poc = df_master[df_master.poc_flag == True]
df_kx_poc.head()

,cid,d1,d10,d2,d3,d4,d5,d6,d7,d8,...,e8,e9,lead,lead_flag,name,num_contestants,poc_flag,season,show,year
335,BA_13_JULIE_D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Jason Mesnick,False,Julie D,25,True,13,Bachelor,2009
360,BA_14_CHANNY_C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Jake Pavelka,False,Channy C,25,True,14,Bachelor,2010
420,BA_16_DIANNA_M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Ben Flajnik,False,Dianna M,25,True,16,Bachelor,2012
425,BA_17_CATHERINE_G,NaN,D1,D13,D12,D8,D8,D1,D3,D1,...,NaN,NaN,Sean Lowe,False,Catherine G,26,True,17,Bachelor,2013
432,BA_17_SELMA_A,NaN,NaN,D13,NaN,D1,D8,D7,NaN,NaN,...,NaN,NaN,Sean Lowe,False,Selma A,26,True,17,Bachelor,2013


In [5]:
# get dataset of people to review
kx_ba_years = range(2009, 2017)
kx_be_years = range(2009, 2016)

is_not_ba_years = ~df_master.year.isin(kx_ba_years) & (df_master.show == "Bachelor")
is_not_be_years = ~df_master.year.isin(kx_be_years) & (df_master.show == "Bachelorette")
is_lead = df_master.lead_flag == True

df_review = df_master[ is_not_ba_years | ( is_not_be_years | is_lead ) ]
df_review.head()

,cid,d1,d10,d2,d3,d4,d5,d6,d7,d8,...,e8,e9,lead,lead_flag,name,num_contestants,poc_flag,season,show,year
0,BA_1_ALEX_M_L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Alex Michel,True,Alex Michel,25,NaN,1,Bachelor,2002
1,BA_01_AMANDA_M,NaN,NaN,D5,D1,D1,D1,D1,NaN,NaN,...,NaN,NaN,Alex Michel,False,Amanda M,25,NaN,1,Bachelor,2002
2,BA_01_TRISTA_R,NaN,NaN,D5,D1,D1,D1,D1,NaN,NaN,...,NaN,NaN,Alex Michel,False,Trista R,25,NaN,1,Bachelor,2002
3,BA_01_SHANNON_O,NaN,NaN,D5,D1,D1,D1,NaN,NaN,NaN,...,NaN,NaN,Alex Michel,False,Shannon O,25,NaN,1,Bachelor,2002
4,BA_01_KIM_X,NaN,NaN,D5,D4,D1,NaN,NaN,NaN,NaN,...,NaN,NaN,Alex Michel,False,Kim X,25,NaN,1,Bachelor,2002


In [6]:
# check there are no already identified POC in df_review
df_review[df_review.poc_flag == True]

,cid,d1,d10,d2,d3,d4,d5,d6,d7,d8,...,e8,e9,lead,lead_flag,name,num_contestants,poc_flag,season,show,year


In [7]:
# save it all!
kx_poc_path = os.path.join(intermed_dir, "karenx_poc.csv")
review_path = os.path.join(intermed_dir, "review_poc.csv")

df_kx_poc.to_csv(kx_poc_path)
df_review.to_csv(review_path)

<a id="us-demographics"></a>
### Grab yearly U.S. based racial demographics data
* It might be interesting to normalize Bachelorette race data with U.S. wide yearly race data.
* **Objective: Pull and import yearly U.S. wide race data for years in dataset; interpolate data for missing years**

In [8]:
# determine which years of race data to source from IPUMS
df_master.year.unique()

array([2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018])

In [9]:
census_dir = os.path.join(input_dir, "race", "census")
census_fls = os.listdir(census_dir)
print(census_fls)

['ACS_05_EST_B03002.txt', 'ACS_05_EST_B03002_metadata.csv', 'ACS_05_EST_B03002_with_ann.csv', 'ACS_06_EST_B03002.txt', 'ACS_06_EST_B03002_metadata.csv', 'ACS_06_EST_B03002_with_ann.csv', 'ACS_07_1YR_B03002.txt', 'ACS_07_1YR_B03002_metadata.csv', 'ACS_07_1YR_B03002_with_ann.csv', 'ACS_08_1YR_B03002.txt', 'ACS_08_1YR_B03002_metadata.csv', 'ACS_08_1YR_B03002_with_ann.csv', 'ACS_09_1YR_B03002.txt', 'ACS_09_1YR_B03002_metadata.csv', 'ACS_09_1YR_B03002_with_ann.csv', 'ACS_10_1YR_B03002.txt', 'ACS_10_1YR_B03002_metadata.csv', 'ACS_10_1YR_B03002_with_ann.csv', 'ACS_11_1YR_B03002.txt', 'ACS_11_1YR_B03002_metadata.csv', 'ACS_11_1YR_B03002_with_ann.csv', 'ACS_12_1YR_B03002.txt', 'ACS_12_1YR_B03002_metadata.csv', 'ACS_12_1YR_B03002_with_ann.csv', 'ACS_13_1YR_B03002.txt', 'ACS_13_1YR_B03002_metadata.csv', 'ACS_13_1YR_B03002_with_ann.csv', 'ACS_14_1YR_B03002.txt', 'ACS_14_1YR_B03002_metadata.csv', 'ACS_14_1YR_B03002_with_ann.csv', 'ACS_15_1YR_B03002.txt', 'ACS_15_1YR_B03002_metadata.csv', 'ACS_15_1Y

In [10]:
# now we have a repository of datasets - time to compile them!
def is_data_file(fl):
    fn, ext = os.path.splitext(fl)
    return "metadata" not in fn and ext == ".csv"

get_fp = lambda fl: os.path.join(census_dir, fl)
    
census_data_fls = list(map(get_fp, filter(is_data_file, census_fls)))
print(census_data_fls)

['../input/race/census/ACS_05_EST_B03002_with_ann.csv', '../input/race/census/ACS_06_EST_B03002_with_ann.csv', '../input/race/census/ACS_07_1YR_B03002_with_ann.csv', '../input/race/census/ACS_08_1YR_B03002_with_ann.csv', '../input/race/census/ACS_09_1YR_B03002_with_ann.csv', '../input/race/census/ACS_10_1YR_B03002_with_ann.csv', '../input/race/census/ACS_11_1YR_B03002_with_ann.csv', '../input/race/census/ACS_12_1YR_B03002_with_ann.csv', '../input/race/census/ACS_13_1YR_B03002_with_ann.csv', '../input/race/census/ACS_14_1YR_B03002_with_ann.csv', '../input/race/census/ACS_15_1YR_B03002_with_ann.csv', '../input/race/census/ACS_16_1YR_B03002_with_ann.csv']


In [11]:
census_dfs = []
for data_file in census_data_fls:
    # expects files named in the format: '../input/race/census/ACS_{YEARSTUB}_1YR_B03002_with_ann.csv'
    year_stub = os.path.basename(data_file).split("_")[1]
    year = int(year_stub) + 2000
    df_year = pd.read_csv(data_file, skiprows=[0])
    
    # filter out margin of error
    df_year = df_year.filter(regex=r"^(?!Margin of Error).*$", axis=1)
    df_year["year"] = year
    df_year = df_year.set_index("year")
    
    census_dfs.append(df_year)
    
df_census = pd.concat(census_dfs)
for colname in df_census.columns:
    print(colname)

df_census.head(2)

Id
Id2
Geography
Estimate; Total:
Estimate; Not Hispanic or Latino:
Estimate; Not Hispanic or Latino: - White alone
Estimate; Not Hispanic or Latino: - Black or African American alone
Estimate; Not Hispanic or Latino: - American Indian and Alaska Native alone
Estimate; Not Hispanic or Latino: - Asian alone
Estimate; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone
Estimate; Not Hispanic or Latino: - Some other race alone
Estimate; Not Hispanic or Latino: - Two or more races:
Estimate; Not Hispanic or Latino: - Two or more races: - Two races including Some other race
Estimate; Not Hispanic or Latino: - Two or more races: - Two races excluding Some other race, and three or more races
Estimate; Hispanic or Latino:
Estimate; Hispanic or Latino: - White alone
Estimate; Hispanic or Latino: - Black or African American alone
Estimate; Hispanic or Latino: - American Indian and Alaska Native alone
Estimate; Hispanic or Latino: - Asian alone
Estimate; Hispanic or Latino:

,Id,Id2,Geography,Estimate; Total:,Estimate; Not Hispanic or Latino:,Estimate; Not Hispanic or Latino: - White alone,Estimate; Not Hispanic or Latino: - Black or African American alone,Estimate; Not Hispanic or Latino: - American Indian and Alaska Native alone,Estimate; Not Hispanic or Latino: - Asian alone,Estimate; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone,...,Estimate; Hispanic or Latino:,Estimate; Hispanic or Latino: - White alone,Estimate; Hispanic or Latino: - Black or African American alone,Estimate; Hispanic or Latino: - American Indian and Alaska Native alone,Estimate; Hispanic or Latino: - Asian alone,Estimate; Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone,Estimate; Hispanic or Latino: - Some other race alone,Estimate; Hispanic or Latino: - Two or more races:,Estimate; Hispanic or Latino: - Two or more races: - Two races including Some other race,"Estimate; Hispanic or Latino: - Two or more races: - Two races excluding Some other race, and three or more races"
year,,,,,,,,,,,,,,,,,,,,,
2005,0100000US,NaN,United States,288378137,246507434,192615561,34364572,2046735,12312949,355513,...,41870703,22717833,597997,310809,158866,41517,16520922,1522759,1030575,492184
2006,0100000US,NaN,United States,299398485,255146207,198176991,36434530,2035551,12945401,387230,...,44252278,23154516,616953,333880,154694,38964,18238347,1714924,1158753,556171


In [12]:
# fix colnames
def get_colname(colname):
    if "Estimate;" in colname:
        colname = re.sub("Estimate; ", "", colname)
    changemap = {
        "Total:": "total",
        "Not Hispanic or Latino:": "nhisp",
        "Not Hispanic or Latino: - White alone": "nhisp_white",
        "Not Hispanic or Latino: - Black or African American alone": "nhisp_afam",
        "Not Hispanic or Latino: - American Indian and Alaska Native alone": "nhisp_amin",
        "Not Hispanic or Latino: - Asian alone": "nhisp_asn",
        "Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone": "nhisp_paci",
        "Not Hispanic or Latino: - Some other race alone": "nhisp_oth",
        "Not Hispanic or Latino: - Two or more races:": "nhisp_mult",
        "Not Hispanic or Latino: - Two or more races: - Two races including Some other race": "nhisp_mult_inc",
        "Not Hispanic or Latino: - Two or more races: - Two races excluding Some other race, and three or more races": "nhisp_mult_exc",
        "Hispanic or Latino:": "hisp",
        "Hispanic or Latino: - White alone": "hisp_white",
        "Hispanic or Latino: - Black or African American alone": "hisp_afam",
        "Hispanic or Latino: - American Indian and Alaska Native alone": "hisp_amin",
        "Hispanic or Latino: - Asian alone": "hisp_asn",
        "Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone": "hisp_paci",
        "Hispanic or Latino: - Some other race alone": "hisp_oth",
        "Hispanic or Latino: - Two or more races:": "hisp_mult",
        "Hispanic or Latino: - Two or more races: - Two races including Some other race": "hisp_mult_inc",
        "Hispanic or Latino: - Two or more races: - Two races excluding Some other race, and three or more races": "hisp_mult_exc"
    }
    return changemap.get(colname, colname.lower())

df_census.columns = map(get_colname, df_census.columns)
df_census = df_census.drop(["id", "id2", "geography"], axis=1)

df_census.head()

,total,nhisp,nhisp_white,nhisp_afam,nhisp_amin,nhisp_asn,nhisp_paci,nhisp_oth,nhisp_mult,nhisp_mult_inc,...,hisp,hisp_white,hisp_afam,hisp_amin,hisp_asn,hisp_paci,hisp_oth,hisp_mult,hisp_mult_inc,hisp_mult_exc
year,,,,,,,,,,,,,,,,,,,,,
2005,288378137,246507434,192615561,34364572,2046735,12312949,355513,777679,4034425,215466,...,41870703,22717833,597997,310809,158866,41517,16520922,1522759,1030575,492184
2006,299398485,255146207,198176991,36434530,2035551,12945401,387230,768782,4397722,231584,...,44252278,23154516,616953,333880,154694,38964,18238347,1714924,1158753,556171
2007,301621159,256193722,198553437,36657280,2019204,13077192,401932,715275,4769402,209750,...,45427437,24452046,677290,346143,156095,32743,18023509,1739611,1147301,592310
2008,304059728,257168272,198942886,36701103,1993622,13239894,402725,701823,5186219,238046,...,46891456,29239524,884947,449800,174082,25085,14290365,1827653,1006533,821120
2009,307006556,258649796,199325978,37144530,1975193,13627633,426897,676733,5472832,236409,...,48356760,30447153,949195,482359,146978,27104,14271630,2032341,1084815,947526


In [13]:
# check whether multi internal columns can be deleted
for stub in ["nhisp", "hisp"]:
    mult_new = "{}_mult_all".format(stub)
    mult_old = "{}_mult".format(stub)
    mult_inc = "{}_mult_inc".format(stub)
    mult_exc = "{}_mult_exc".format(stub)
    
    df_census[mult_new] = df_census[mult_inc] + df_census[mult_exc]
    any_mismatch = df_census[df_census[mult_new] != df_census[mult_old]]
    if len(any_mismatch) == 0:
        df_census = df_census.drop([mult_new, mult_inc, mult_exc], axis=1)
        print("Success!")
print(df_census.columns)

Success!
Success!
Index(['total', 'nhisp', 'nhisp_white', 'nhisp_afam', 'nhisp_amin',
       'nhisp_asn', 'nhisp_paci', 'nhisp_oth', 'nhisp_mult', 'hisp',
       'hisp_white', 'hisp_afam', 'hisp_amin', 'hisp_asn', 'hisp_paci',
       'hisp_oth', 'hisp_mult'],
      dtype='object')


While the question of how the Census collects data on Hispanics/Latino/a is [somewhat controversial](http://www.pewsocialtrends.org/2010/03/03/census-history-counting-hispanics-2/), for the purpose of this investigation, we can treat "Hispanic" as a distinct ethnic/racial category to keep things simple.

In [14]:
# consolidate categories where possible
df_census = df_census.drop(
    ["hisp_white", "hisp_afam", "hisp_amin", "hisp_asn", "hisp_paci", "hisp_oth", "hisp_mult", "nhisp"], axis=1)

df_census.columns = map(lambda nm: re.sub("nhisp_", "", nm), df_census.columns)

df_census["asn_paci"] = df_census.asn + df_census.paci
df_census = df_census.drop(["asn", "paci"], axis=1)
df_census["nwhite"] = df_census.afam + df_census.amin + df_census.oth + df_census.mult + df_census.hisp + df_census.asn_paci

df_census.head()

,total,white,afam,amin,oth,mult,hisp,asn_paci,nwhite
year,,,,,,,,,
2005,288378137,192615561,34364572,2046735,777679,4034425,41870703,12668462,95762576
2006,299398485,198176991,36434530,2035551,768782,4397722,44252278,13332631,101221494
2007,301621159,198553437,36657280,2019204,715275,4769402,45427437,13479124,103067722
2008,304059728,198942886,36701103,1993622,701823,5186219,46891456,13642619,105116842
2009,307006556,199325978,37144530,1975193,676733,5472832,48356760,14054530,107680578


In [15]:
# grab percentile data
for col in filter(lambda x: x != "total", df_census.columns):
    df_census["{}_perc".format(col)] = df_census[col] / df_census.total

df_census.head()

,total,white,afam,amin,oth,mult,hisp,asn_paci,nwhite,white_perc,afam_perc,amin_perc,oth_perc,mult_perc,hisp_perc,asn_paci_perc,nwhite_perc
year,,,,,,,,,,,,,,,,,
2005,288378137,192615561,34364572,2046735,777679,4034425,41870703,12668462,95762576,0.667927,0.119165,0.007097,0.002697,0.013990,0.145194,0.043930,0.332073
2006,299398485,198176991,36434530,2035551,768782,4397722,44252278,13332631,101221494,0.661917,0.121692,0.006799,0.002568,0.014689,0.147804,0.044531,0.338083
2007,301621159,198553437,36657280,2019204,715275,4769402,45427437,13479124,103067722,0.658287,0.121534,0.006695,0.002371,0.015813,0.150611,0.044689,0.341713
2008,304059728,198942886,36701103,1993622,701823,5186219,46891456,13642619,105116842,0.654289,0.120704,0.006557,0.002308,0.017057,0.154218,0.044868,0.345711
2009,307006556,199325978,37144530,1975193,676733,5472832,48356760,14054530,107680578,0.649256,0.120989,0.006434,0.002204,0.017826,0.157511,0.045779,0.350744
